# Answers:


1. 0.6798  accuracy with threshold = 2 on validation set
2. 0.69845 accuracy with threshold = 1.58 on validation set
3. 0.67395 accuracy with threshold 0.031 on validation set
4. 0.70325 accuracy with 0.022 jaccard threshold and  1.45 popularity threshold on validation set
5. Username: tegrity
6. [(544597, 'the'), (317620, 'and'), (305414, 'a'), (291882, 'to'), (245359, 'game'), (227234, 'of'), (208417, 'is'), (200633, 'you'), (195953, 'i'), (190966, 'it')]
7. 0.6722 accuracy on validation set
8. 0.6844 accuracy with C = 10 and dictionary size of 1200

# Source Code:

## Setting Up

In [3]:
import gzip
import numpy
from matplotlib import pyplot as plt
import random
import numpy as np
import tarfile
from collections import defaultdict
import json
import statistics
import string
from sklearn.linear_model import LogisticRegression
import math

In [4]:
#f = tarfile.open("assignment1.tar.gz", 'r')
#f.getnames()
#f.extractall()

In [5]:
dataset = []
lines = []

f_2 = gzip.open("train.json.gz", 'r+')

for line in f_2:
    value = eval(line)
    dataset.append(value)
f_2.close()
print(len(dataset))       

175000


In [ ]:
dataset[4]

In [6]:
#random.shuffle(dataset)
N = 165000
temp = [(d['userID'],d['gameID']) for d in dataset]
training = temp[:N]
validation = temp[N:]
print(len(training))
print(len(validation))

165000
10000


In [7]:
gamesPerUser_1 = defaultdict(set)
all_available_games = set()



for d in dataset:
    user,game, = d['userID'], d['gameID']
    gamesPerUser_1[user].add(game)
    all_available_games.add(game)
    



## Question 1-2

In [8]:
negatives= []

for item in validation:
    user = item[0]
    games_played = gamesPerUser_1[user]
    games_not_played = all_available_games.difference(games_played)
    random_choice = random.choice(list(games_not_played))
    negatives.append((user,random_choice))

validation = validation + negatives 
    

In [7]:
print(validation)

[('u49969792', 'b25961467'), ('u33147591', 'b73229067'), ('u00954406', 'b37068085'), ('u40416473', 'b85572219'), ('u08125051', 'b88846011'), ('u16794935', 'b19939295'), ('u42345370', 'b31288876'), ('u40134397', 'b22332890'), ('u38053603', 'b44539786'), ('u25257262', 'b64105731'), ('u04355033', 'b67179888'), ('u40085824', 'b09213636'), ('u66137217', 'b72051448'), ('u60516559', 'b82987255'), ('u79477484', 'b59467286'), ('u65083695', 'b36061876'), ('u70957097', 'b04084075'), ('u35234877', 'b34733510'), ('u67210927', 'b40652749'), ('u71295643', 'b10009358'), ('u14505832', 'b60633667'), ('u37899313', 'b08749860'), ('u24610650', 'b24086450'), ('u70460581', 'b73525270'), ('u56233169', 'b07673761'), ('u25967030', 'b45313277'), ('u22455223', 'b06690738'), ('u70829339', 'b26913090'), ('u38913894', 'b84695943'), ('u69944776', 'b69475426'), ('u66482270', 'b80454010'), ('u64964763', 'b10986469'), ('u91283414', 'b90916347'), ('u93839419', 'b01959849'), ('u59111590', 'b85805582'), ('u05832068', 'b360

In [49]:
def popularity_threshold(threshold):
    
    gameCount = defaultdict(int)
    totalPlayed = 0

    for user,game in training:
        gameCount[game] += 1
        totalPlayed += 1
  

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed/threshold: 
            print(count)
            break
            

    predictions_popularity = []
    for l in validation:
        u,g = l[0],l[1]
        if g in return1:
            predictions_popularity.append(1)
        else:
            predictions_popularity.append(0)

    
    return predictions_popularity

In [50]:
best_thresholdd = []
for j in range(1,101):
    popularity_results = popularity_threshold(1+j/100) 
    total_errors = 0
    for i in range(10000):
        if popularity_results[i] == 0:
            total_errors += 1
        
    for i in range(10000,20000):
        if popularity_results[i] == 1:
            total_errors += 1
        
    acc = 1-(total_errors/20000)
    best_thresholdd.append((acc,j/100))

163377
161770
160203
158661
157158
155664
154211
152798
151377
150012
148658
147332
146042
144761
143496
142252
141033
139843
138683
137513
136382
135257
134180
133069
132013
130969
129941
128921
127922
126947
125998
125030
124095
123144
122226
121343
120439
119568
118735
117885
117073
116246
115402
114601
113848
113077
112290
111491
110743
110047
109341
108620
107888
107146
106471
105788
105097
104476
103843
103201
102549
101888
101303
100624
100023
99417
98808
98276
97650
97109
96561
96008
95448
94884
94314
93833
93249
92756
92261
91762
91258
90750
90236
89715
89295
88765
88334
87791
87351
86910
86463
86013
85558
85102
84642
84295
83827
83352
82994
82512


In [11]:
best_thresholdd.sort(reverse=True)
best_thresholdd[0]

(0.70345, 0.53)

## Question 3

In [12]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom
 

In [13]:
usersPerGame = defaultdict(set)
gamesPerUser_2 = defaultdict(set)
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
for d in training:
    user,game = d[0], d[1]
    gamesPerUser_2[user].add(game)
    usersPerGame[game].add(user)

for d in dataset:
    user, game = d['userID'],d['gameID']
    r = d['hours_transformed']
    hoursPerUser[user].append((game,r))
    hoursPerItem[game].append((user,r))

In [14]:
def jaccard_threshold(threshold):
    
    predictions_jaccard = []
    for element in mean_similarity:    
        if element > threshold:
            predictions_jaccard.append(1)
        else:
            predictions_jaccard.append(0)
    
    return predictions_jaccard

In [15]:
mean_similarity = []
for item in validation:
    user = item[0]
    g = item[1]
    similarities = []
    g_prime = gamesPerUser_2[user]
    for prime in g_prime:
        if g == prime: continue
        sim = Jaccard(usersPerGame[g],usersPerGame[prime])
        similarities.append(sim)      
    mean_similarity.append(statistics.mean(similarities))

In [16]:
jaccard_results = jaccard_threshold(0.0094)
total_errors = 0
for i in range(10000):
    if jaccard_results[i] == 0:
        total_errors += 1
for i in range(10000,19999):
    if jaccard_results[i] == 1:
        total_errors += 1
            
acc = 1-(total_errors/20000)
acc

0.7135

In [17]:
def Cosine(s1, s2):
    numer = abs(len(s1.intersection(s2)))
    denom = math.sqrt(abs(len(s1))*abs(len(s2)))
    return numer / denom

In [18]:
mean_cos = []
for item in validation:
    user = item[0]
    g = item[1]
    cos_similar = []
    g_prime = gamesPerUser_2[user]
    for prime in g_prime:
        if g == prime: continue
        angle = Cosine(usersPerGame[g],usersPerGame[prime])
        cos_similar.append(angle)      
    mean_cos.append(statistics.mean(cos_similar))

In [19]:
def cosine_threshold(threshold):
    
    predictions_cosine = []
    for element in mean_cos:    
        if element > threshold:
            predictions_cosine.append(1)
        else:
            predictions_cosine.append(0)
    
    return predictions_cosine

In [20]:
cosine_results = cosine_threshold(0.0222)
total_errors = 0
for i in range(10000):
    if cosine_results[i] == 0:
        total_errors += 1
for i in range(10000,19999):
    if cosine_results[i] == 1:
        total_errors += 1

acc = 1-(total_errors/20000)
acc

0.7072

In [62]:
gg = []
ff = []
for u,g in validation[11:12]:
    if len(hoursPerItem[g]) > 25:
        print((hoursPerItem[g]))
        
        gg.append(1)
        ff.append(len(hoursPerItem[g]))
    else:
        gg.append(0)
        ff.append(len(hoursPerItem[g]))

[('u29694608', 2.321928094887362), ('u37919673', 4.694880192799191), ('u80286207', 5.116863757690899), ('u77337833', 2.2016338611696504), ('u03744953', 1.3219280948873624), ('u70540747', 7.2393598502774665), ('u23825527', 8.306517445249819), ('u48727616', 9.502831804066744), ('u77396791', 8.0405637111587), ('u38147884', 1.4329594072761063), ('u80937699', 1.6780719051126378), ('u81773016', 5.22881869049588), ('u96671815', 4.916476644437717), ('u18489131', 3.2172307162206693), ('u13682966', 4.8529975876133165), ('u43030076', 5.730639955916791), ('u17812661', 10.692179576312538), ('u27496659', 5.343407822297814), ('u65028243', 2.035623909730721), ('u93497970', 2.035623909730721), ('u96372118', 2.035623909730721), ('u84689253', 5.104336659814735), ('u22430702', 2.7224660244710908), ('u78730689', 5.452858964713811), ('u53640885', 1.6780719051126378), ('u82252219', 0.48542682717024166), ('u80739155', 3.608809242675524), ('u82273767', 7.1947568544222475), ('u02760195', 4.523561956057013), ('u

In [ ]:
len(gg)

In [ ]:
total_errors = 0
for i in range(10000):
    if gg[i] == 0:
        total_errors += 1
for i in range(10000,19999):
    if gg[i] == 1:
        total_errors += 1

acc = 1-(total_errors/20000)
acc

In [ ]:
statistics.median(ff)

## Question 4


In [ ]:
ensemble_results = []

            
best_combo = []
jaccard = jaccard_threshold(0.0094)
cosine = cosine_threshold(0.0229)
popularity = popularity_threshold(1.53)
    
for i in range(20000):
    if popularity[i] == 0 and jaccard[i] == 0 and cosine[i] == 0 and gg[i] == 0:
        best_combo.append(0)
    elif popularity[i] == 0 and jaccard[i] == 0 and cosine[i] == 0 and gg[i] == 1:
        best_combo.append(0)
    elif popularity[i] == 0 and jaccard[i] == 0 and cosine[i] == 1 and gg[i] == 0:
        best_combo.append(0)
    elif popularity[i] == 0 and jaccard[i] == 0 and cosine[i] == 1 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 0 and jaccard[i] == 1 and cosine[i] == 0 and gg[i] == 0:
        best_combo.append(0)
    elif popularity[i] == 0 and jaccard[i] == 1 and cosine[i] == 0 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 0 and jaccard[i] == 1 and cosine[i] == 1 and gg[i] == 0:
        best_combo.append(1)
    elif popularity[i] == 0 and jaccard[i] == 1 and cosine[i] == 1 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 0 and cosine[i] == 0 and gg[i] == 0:
        best_combo.append(0)
    elif popularity[i] == 1 and jaccard[i] == 0 and cosine[i] == 0 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 0 and cosine[i] == 1 and gg[i] == 0:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 0 and cosine[i] == 1 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 1 and cosine[i] == 0 and gg[i] == 0:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 1 and cosine[i] == 0 and gg[i] == 1:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 1 and cosine[i] == 1 and gg[i] == 0:
        best_combo.append(1)
    elif popularity[i] == 1 and jaccard[i] == 1 and cosine[i] == 1 and gg[i] == 1:
        best_combo.append(1)                
total_errors = 0
for i in range(10000):
    if best_combo[i] == 0:
        total_errors += 1
for i in range(10000,20000):
    if best_combo[i] == 1:
        total_errors += 1

acc = 1-(total_errors/20000)
            
ensemble_results.append(acc)
            

In [ ]:
ensemble_results.sort(reverse=True)
print(ensemble_results[:10])

## Question 5

In [41]:

def readJSON(path):
    for l in gzip.open(path, 'r+'):
        d = eval(l)
        u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d

gameCount = defaultdict(int)
totalPlayed = 0

for user,game in training:
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/1.45: break
        
predictions_popularity = []
for l in open("pairs_Played.txt"):
    if l.startswith("userID"): continue
    u,g = l.strip().split('-')
    if g in return1:
        predictions_popularity.append(1)
    else:
        predictions_popularity.append(0)


In [42]:
mean_similarity = []
for item in open("pairs_Played.txt"):
    if item.startswith("userID"): continue
    user, g = item.strip().split('-')
    similarities = []
    g_prime = gamesPerUser_2[user]
    for prime in g_prime:
        if g == prime: continue
        sim = Jaccard(usersPerGame[g],usersPerGame[prime] )
        similarities.append(sim)         
    if len(similarities) == 0: 
        mean_similarity.append(0) 
    else: 
        mean_similarity.append(statistics.mean(similarities))

In [43]:
mean_cos = []
for item in open("pairs_Played.txt"):
    if item.startswith("userID"): continue
    user, g = item.strip().split('-')
    cos_similar = []
    g_prime = gamesPerUser_2[user]
    for prime in g_prime:
        if g == prime: continue
        angle = Cosine(usersPerGame[g],usersPerGame[prime])
        cos_similar.append(angle)
    if len(cos_similar) == 0:
        mean_cos.append(0)
    else:
        mean_cos.append(statistics.mean(cos_similar))

In [44]:
predictions_jaccard = []
for element in mean_similarity:    
    if element > 0.0094:
        predictions_jaccard.append(1)
    else:
        predictions_jaccard.append(0)

In [45]:
predictions_cosine = []
for element in mean_cos:    
    if element > 0.0222:
        predictions_cosine.append(1)
    else:
        predictions_cosine.append(0)
    

In [51]:
predictions_hours = []
for l in open("pairs_Played.txt"):
    if l.startswith("userID"):
        continue
    u,g = l.strip().split('-')
    maxSim = 0
    users = set(hoursPerItem[g])
    for g2,_ in hoursPerUser[u]:
        sim = Jaccard(users,set(hoursPerItem[g2]))
        if sim > maxSim:
            maxSim = sim
    pred = 0
    if len(hoursPerItem[g]) > 325:
        predictions_hours.append(1)
    else:
        predictions_hours.append(0)

In [52]:
final_prediction = []

    
for i in range(20000):
    if predictions_popularity[i] == 0 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 0 and predictions_hours[i] == 0:
        final_prediction.append(0)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 0 and predictions_hours[i] == 1:
        final_prediction.append(0)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 1 and predictions_hours[i] == 0:
        final_prediction.append(0)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 1 and predictions_hours[i] == 1:
        final_prediction.append(1)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 0 and predictions_hours[i] == 0:
        final_prediction.append(0)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 0 and predictions_hours[i] == 1:
        final_prediction.append(1)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 1 and predictions_hours[i] == 0:
        final_prediction.append(1)
    elif predictions_popularity[i] == 0 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 1 and predictions_hours[i] == 1:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 0 and predictions_hours[i] == 0:
        final_prediction.append(0)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 0 and predictions_hours[i] == 1:
        final_prediction.append(0)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 1 and predictions_hours[i] == 0:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 0 and predictions_cosine[i] == 1 and predictions_hours[i] == 1:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 0 and predictions_hours[i] == 0:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 0 and predictions_hours[i] == 1:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 1 and predictions_hours[i] == 0:
        final_prediction.append(1)
    elif predictions_popularity[i] == 1 and predictions_jaccard[i] == 1 and predictions_cosine[i] == 1 and predictions_hours[i] == 1:
        final_prediction.append(1)                

        
        
            

In [53]:
print(final_prediction)


[1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 

In [54]:
predictions = open("predictions_Played.txt", 'w')
for l in open("pairs_Played.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    if final_prediction[0] == 1:
        predictions.write(u + '-' + g + ",1\n")
    else:
        predictions.write(u + '-' + g + ",0\n")
    del final_prediction[0]

predictions.close()

        


In [1]:
bbb = [('the',7),('fuccc',8)]

In [2]:
print(len(bbb))

2
